# TDSP Stage 4 – Deployment & Business Insights

## Proyek: Segmentasi Pelanggan Online Retail Berbasis RFM (Spark)

Stage ini berfokus pada **interpretasi hasil modeling**, **penerjemahan segmentasi ke insight bisnis**,
serta **penyusunan output yang siap digunakan oleh stakeholder**.

## 4.1 Tujuan Deployment

Tujuan utama tahap deployment adalah:

1. Menyajikan hasil segmentasi pelanggan dalam bentuk yang mudah dipahami.
2. Mengaitkan setiap cluster dengan karakteristik bisnis yang jelas.
3. Menghasilkan insight yang dapat langsung ditindaklanjuti (*actionable*).
4. Menyiapkan artefak analitik yang siap diintegrasikan ke proses bisnis.


## 4.2 Ringkasan Segmentasi Pelanggan

Segmentasi pelanggan dihasilkan menggunakan **K-Means (k = 5)** berdasarkan fitur RFM.

Setiap cluster merepresentasikan **pola perilaku transaksi yang berbeda**, sehingga strategi bisnis
tidak dapat disamaratakan untuk seluruh pelanggan.

In [ ]:
# ==============================================================
# 4.2 Load Hasil Clustering dari Stage 3
# ==============================================================
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("RFM-Stage4-Deployment").getOrCreate()

# Load hasil clustering
rfm_clustered = spark.read.parquet("../datasets/rfm_clustered.parquet")

print(f"Total Customers: {rfm_clustered.select('CustomerID').count()}")
rfm_clustered.select("CustomerID", "prediction").show(5)


## 4.3 Profil RFM per Cluster

Untuk memahami karakteristik tiap segmen, dilakukan agregasi statistik RFM
pada level cluster.

Profil ini menjadi dasar utama dalam menentukan **strategi marketing**.

In [ ]:
# ==============================================================
# 4.3 Profil RFM per Cluster
# ==============================================================
from pyspark.sql.functions import avg, count, round as spark_round

cluster_profile = (
    rfm_clustered
    .groupBy("prediction")
    .agg(
        count("CustomerID").alias("Num_Customers"),
        spark_round(avg("Recency"), 1).alias("Avg_Recency"),
        spark_round(avg("Frequency"), 1).alias("Avg_Frequency"),
        spark_round(avg("Monetary"), 2).alias("Avg_Monetary")
    )
    .orderBy("prediction")
)

cluster_profile.show()


## 4.4 Interpretasi Bisnis per Cluster

Berdasarkan profil RFM, cluster dapat diinterpretasikan secara konseptual sebagai berikut:

- **Cluster 0 – Low Value / Dormant Customers**  
  Recency tinggi, Frequency rendah, Monetary rendah. Pelanggan jarang bertransaksi dan berisiko churn.

- **Cluster 1 – Occasional Buyers**  
  Frequency rendah–menengah dengan nilai transaksi terbatas.

- **Cluster 2 – Regular Customers**  
  Pelanggan dengan pola belanja stabil dan kontribusi moderat.

- **Cluster 3 – High Value Customers**  
  Frequency dan Monetary tinggi, memberikan kontribusi signifikan terhadap revenue.

- **Cluster 4 – Very High Value / VIP Customers**  
  Pelanggan paling bernilai, jumlah kecil namun kontribusi revenue sangat besar.


In [ ]:
# ==============================================================
# 4.4 Mapping Cluster → Business Segment
# ==============================================================
from pyspark.sql.functions import when

rfm_labeled = (
    rfm_clustered
    .withColumn(
        "Segment_Label",
        when(col("prediction") == 0, "Low Value / Dormant")
        .when(col("prediction") == 1, "Occasional Buyers")
        .when(col("prediction") == 2, "Regular Customers")
        .when(col("prediction") == 3, "High Value Customers")
        .when(col("prediction") == 4, "Very High Value / VIP")
        .otherwise("Unknown")
    )
)

rfm_labeled.select(
    "CustomerID", "Recency", "Frequency", "Monetary", "Segment_Label"
).show(10)


## 4.5 Rekomendasi Strategi Bisnis

Berdasarkan segmentasi pelanggan, rekomendasi strategis yang dapat diterapkan:

### Cluster Low Value / Dormant
- Kampanye reaktivasi (voucher, reminder)
- Evaluasi biaya marketing (hindari overspending)

### Cluster Regular Customers
- Upselling dan cross-selling
- Program loyalitas ringan

### Cluster High & Very High Value
- Program VIP / loyalty eksklusif
- Personalisasi penawaran
- Prioritas layanan pelanggan

Pendekatan ini memungkinkan perusahaan memaksimalkan ROI dari aktivitas marketing.

In [ ]:
# ==============================================================
# 4.5 Segment Distribution (Untuk Prioritas Strategi)
# ==============================================================
segment_distribution = (
    rfm_labeled
    .groupBy("Segment_Label")
    .agg(count("CustomerID").alias("Num_Customers"))
    .orderBy(col("Num_Customers").desc())
)

segment_distribution.show()


## 4.6 Artefak Deployment

Output dari analisis ini dapat dideploy dalam berbagai bentuk:

- Tabel segmentasi pelanggan (PostCode → Cluster)
- Dashboard BI (Power BI / Tableau / Fabric)
- Dataset input untuk kampanye marketing
- Dasar untuk analisis lanjutan (CLV, churn prediction)


In [ ]:
# ==============================================================
# Contoh artefak: dataset segmentasi pelanggan
# ==============================================================
customer_segments = rfm_clustered.select(
    "PostCode", "Recency", "Frequency", "Monetary", "cluster"
)

customer_segments.show(10)

## Ringkasan Stage 4

Pada TDSP Stage 4 ini, kita telah:
- Menginterpretasikan hasil clustering pelanggan
- Mengaitkan cluster dengan konteks dan strategi bisnis
- Menyiapkan artefak analitik untuk deployment

Tahap terakhir adalah **TDSP Stage 5 – Customer Acceptance**,
yang berfokus pada validasi hasil bersama stakeholder dan perencanaan tindak lanjut.